In [5]:
import json

def load_json(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

def save_json(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, indent=4, ensure_ascii=False)

def prepare_cross_encoder_dataset(json_data):
    dataset_samples = []
    for item in json_data:
        question = item['Question']

        # Process correct and similar answers with a score of 1
        relevant_contexts = [item['Answer']] + item['Similar_answers']
        for context in relevant_contexts:
            sample = {
                'query': question,
                'context': context,
                'score': 1
            }
            dataset_samples.append(sample)

        # Process poor answers with a score of 0
        for context in item['Poor_answers']:
            sample = {
                'query': question,
                'context': context,
                'score': 0
            }
            dataset_samples.append(sample)

    return dataset_samples

# Load data
json_data = load_json('../data/Final_QA_pairs.json')

# Prepare dataset
cross_encoder_dataset = prepare_cross_encoder_dataset(json_data)

# Save the prepared dataset
save_json(cross_encoder_dataset, '../data/Cross_Encoder_Finetuning_Dataset.json')

print("Dataset prepared and saved successfully!")


Dataset prepared and saved successfully!


In [22]:
dataset = load_json('../data/Cross_Encoder_Finetuning_Dataset.json')
from dataclasses import dataclass
@dataclass
class CrossEncoderFinetuningDatasetSample:
    query: str
    context: str
    score: int
tuning_dataset = []
for i in dataset:
    question_row = CrossEncoderFinetuningDatasetSample(
        query=i["query"], context=i["context"], score=i["score"]
    )
    tuning_dataset.append(question_row)

print(len(dataset))

3850


In [26]:
%pip install llama-index-finetuning
%pip install llama-index-llms-openai


[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
from llama_index.finetuning.cross_encoders import CrossEncoderFinetuneEngine


finetuning_dataset = dataset

#from sentence_transformers import SentenceTransformer

# Initialise the cross-encoder fine-tuning engine
finetuning_engine = CrossEncoderFinetuneEngine(
    dataset=finetuning_dataset, epochs=2, batch_size=8
)

# Finetune the cross-encoder model
finetuning_engine.finetune()

ModuleNotFoundError: No module named 'llama_index.llms.azure_openai'

In [10]:
import pandas as pd
import os
import json
from llama_index.core import Document
from llama_index.finetuning.cross_encoders import CrossEncoderFinetuneEngine
